# Importing required libraries 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

# Load the data

In [14]:

exp_df = pd.read_csv("C:/Users/39351/github/Eu_healthcare_spending_analysis/data/raw/eu_health_expending.csv", compression='gzip')
life_df = pd.read_csv("C:/Users/39351/github/Eu_healthcare_spending_analysis/data/raw/life-expectancy.csv", compression='gzip')
mort_df = pd.read_csv("C:/Users/39351/github/Eu_healthcare_spending_analysis/data/raw/mortality_rate.csv", compression='gzip')


# Cleaning the Data

In [47]:

exp_df.head()
exp_df.columns, exp_df.shape
exp_df['Classification of health care financing schemes - SHA 2011'].value_counts()

# remove unnecessary columns
exp_clean = exp_df[['geo',
        'Geopolitical entity (reporting)', 'TIME_PERIOD', 'unit', 'icha11_hf','Classification of health care financing schemes - SHA 2011', 'OBS_VALUE']]


# renaming the columns
exp_clean.rename(columns={
    'geo': 'Country Code',
    'Geopolitical entity (reporting)': 'Country Name', 
    'TIME_PERIOD': 'Year', 
    'unit': 'Unit', 
    'icha11_hf': 'Financing Code',
    'Classification of health care financing schemes - SHA 2011': 'Category', 
    'OBS_VALUE': 'Spending'
}, inplace=True)

exp_clean['Unit'].value_counts()
exp_clean.head()

C:\Users\39351\AppData\Local\Temp\ipykernel_26396\4270225949.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp_clean.rename(columns={


,Country Code,Country Name,Year,Unit,Financing Code,Category,Spending
0,AT,Austria,2012,EUR_HAB,HF121,Social health insurance schemes,1678.79
1,AT,Austria,2013,EUR_HAB,HF121,Social health insurance schemes,1711.40
2,AT,Austria,2014,EUR_HAB,HF121,Social health insurance schemes,1770.42
3,AT,Austria,2015,EUR_HAB,HF121,Social health insurance schemes,1818.96
4,AT,Austria,2016,EUR_HAB,HF121,Social health insurance schemes,1866.41


In [57]:
life_df.head()
life_df.columns, life_df.shape

#remove unnecessary information
life_clean = life_df[['geo', 'Geopolitical entity (reporting)', 'Unit of measure', 
                      'sex', 'age', 'Age class', 
                      'TIME_PERIOD', 'OBS_VALUE']]

# rename columns
life_clean.rename(columns={
    'geo': 'Country Code', 
    'Geopolitical entity (reporting)': 'Country Name', 
    'Unit of measure': 'unit',
    'sex': 'Gender', 
    'age': 'Age', 
    'Age class': 'Age Class',
    'TIME_PERIOD': 'Year', 
    'OBS_VALUE': 'Life Expectancy'  
}, inplace=True)

life_clean.head()
# life_clean['Age'].value_counts()


C:\Users\39351\AppData\Local\Temp\ipykernel_26396\4271783066.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  life_clean.rename(columns={


,Country Code,Country Name,unit,Gender,Age,Age Class,Year,Life Expectancy
0,AL,Albania,Year,F,Y1,1 year,2013,79.6
1,AL,Albania,Year,F,Y1,1 year,2014,79.8
2,AL,Albania,Year,F,Y1,1 year,2015,79.2
3,AL,Albania,Year,F,Y1,1 year,2016,79.8
4,AL,Albania,Year,F,Y1,1 year,2017,79.7


In [56]:
mort_df.head()
mort_df.columns, mort_df.shape

# remove unnecessary columns
mort_clean = mort_df[['geo', 'Geopolitical entity (reporting)','Type of mortality', 
                      'sex', 'icd10','Unit of measure','TIME_PERIOD', 'OBS_VALUE']]

mort_clean.rename(columns={
    'geo': 'Country Code', 
    'Geopolitical entity (reporting)': 'Country Name',
    'Type of mortality': 'Type of Mortality', 
    'sex': 'Gender', 
    'icd10': 'Diseas',
    'Unit of measure': 'Unit',
    'TIME_PERIOD': 'Year', 
    'OBS_VALUE': 'Mortality Rate'
}, inplace=True)

mort_clean.head()

C:\Users\39351\AppData\Local\Temp\ipykernel_26396\2593528078.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mort_clean.rename(columns={


,Country Code,Country Name,Type of Mortality,Gender,Diseas,Unit,Year,Mortality Rate
0,AT,Austria,Preventable mortality,F,TOTAL,Rate,2012,96.16
1,AT,Austria,Preventable mortality,F,TOTAL,Rate,2013,93.22
2,AT,Austria,Preventable mortality,F,TOTAL,Rate,2014,93.95
3,AT,Austria,Preventable mortality,F,TOTAL,Rate,2015,97.52
4,AT,Austria,Preventable mortality,F,TOTAL,Rate,2016,96.39
